In [ ]:
 !pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 9.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
data = yf.download('BTC-USD', start='2022-01-01')

[*********************100%***********************]  1 of 1 completed


In [ ]:
df = data.copy()

In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-01 00:00:00+00:00,46311.746094,47827.312500,46288.484375,47686.812500,47686.812500,24582667004
2022-01-02 00:00:00+00:00,47680.925781,47881.406250,46856.937500,47345.218750,47345.218750,27951569547
2022-01-03 00:00:00+00:00,47343.542969,47510.726562,45835.964844,46458.117188,46458.117188,33071628362
2022-01-04 00:00:00+00:00,46458.851562,47406.546875,45752.464844,45897.574219,45897.574219,42494677905
2022-01-05 00:00:00+00:00,45899.359375,46929.046875,42798.222656,43569.003906,43569.003906,36851084859
...,...,...,...,...,...,...
2023-01-13 00:00:00+00:00,18868.906250,19964.322266,18753.164062,19909.574219,19909.574219,29225029694
2023-01-14 00:00:00+00:00,19910.537109,21075.142578,19907.828125,20976.298828,20976.298828,38967784639
2023-01-15 00:00:00+00:00,20977.484375,20993.748047,20606.986328,20880.798828,20880.798828,19298407543


In [ ]:
df.dtypes

Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [ ]:
def macd_func(df,a=12,b=21,c=9):
  EMA_short = df['Close'].ewm(span=a, adjust=False,min_periods= a+1).mean()
  EMA_long = df['Close'].ewm(span=b, adjust=False,min_periods= b+1).mean()
  MACD = EMA_short - EMA_long
  signal = MACD.ewm(span=c, adjust=False,min_periods=c+1).mean()
  df['macd'] = MACD
  df['signal_line'] = signal
  df['histogram'] = df['macd'] - df['signal_line']
  
  #find entry point
  df['shift_hist'] = df['histogram'].shift(1)
  df[(df['histogram'] > 0) & (df['shift_hist'] < 0)]
  df.loc[(df['histogram']> 0) &  (df['shift_hist'] < 0 ), 'action'] = 'buy'
  df.loc[(df['histogram']< 0) &  (df['shift_hist'] > 0 ), 'action'] = 'sell'
  df.loc[df['action'] == 'buy', 'marker_position'] = df['Low'] *0.98
  df.loc[df['action'] == 'sell', 'marker_position'] = df['High'] *1.02


  return df

In [ ]:
def strategy_return(df, action_price = 'Open', long_or_short='long'):
  
  action_df = df.loc[df['action'].notnull()]

  if action_price == 'Open':

    df['action_price'] = df['Open'].shift(-1)
    action_df['return'] = action_df['action_price'].pct_change()


  elif action_price == 'Close':
    df['action_price'] = df['Close']
    action_df['return'] = action_df['action_price'].pct_change()

  elif action_price == 'Adj Close':
    df['action_price'] = df['Adj Close']
    action_df['return'] = action_df['Adj Close'].pct_change()

  else:
    raise ValueError('Not correct action price, choose between {Open, Close, Adj Close}')

  
  record_df = action_df[['action','action_price','return']]
  record_df.reset_index(inplace=True)
  record_df['previous_signal_date'] = record_df['Date'].shift(1)
  record_df['previous_action'] = record_df['action'].shift(1)
  record_df['previous_action_price'] = record_df['action_price'].shift(1)
  record_df.rename(columns={'Date':'signal_date'}, inplace=True)
  

  if long_or_short == 'long':
    summary_df = record_df[record_df.action == 'sell']
    print('long strategy')
  
  elif long_or_short == 'short':
    summary_df = record_df[record_df.action == 'buy']
    summary_df['return'] = summary_df['return'] * -1
    print('short strategy')

  else:
    raise ValueError('Not correct long_or_short, choose between {long, short}')

  return action_df, record_df, summary_df

In [ ]:
df.dtypes

Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [ ]:
    starter = macd_func(df)

In [ ]:
starter

,Open,High,Low,Close,Adj Close,Volume,macd,signal_line,histogram,shift_hist,action,marker_position
Date,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,46311.746094,47827.312500,46288.484375,47686.812500,47686.812500,24582667004,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-02 00:00:00+00:00,47680.925781,47881.406250,46856.937500,47345.218750,47345.218750,27951569547,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03 00:00:00+00:00,47343.542969,47510.726562,45835.964844,46458.117188,46458.117188,33071628362,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04 00:00:00+00:00,46458.851562,47406.546875,45752.464844,45897.574219,45897.574219,42494677905,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05 00:00:00+00:00,45899.359375,46929.046875,42798.222656,43569.003906,43569.003906,36851084859,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-13 00:00:00+00:00,18868.906250,19964.322266,18753.164062,19909.574219,19909.574219,29225029694,350.435170,113.719704,236.715465,157.137133,NaN,NaN
2023-01-14 00:00:00+00:00,19910.537109,21075.142578,19907.828125,20976.298828,20976.298828,38967784639,519.406799,194.857123,324.549675,236.715465,NaN,NaN
2023-01-15 00:00:00+00:00,20977.484375,20993.748047,20606.986328,20880.798828,20880.798828,19298407543,636.110055,283.107710,353.002345,324.549675,NaN,NaN


In [ ]:
a,b,c = strategy_return(starter,'Open','long')

KeyError: ignored

In [ ]:
c

In [ ]:
c['return'].plot(kind='bar',figsize=(12,8))

In [ ]:
c.mean()